In [8]:
import os
import cv2
import numpy as np
import pandas as pd

In [9]:
base_path = '/home/simon/Data/pAVK_dataset/'
files = os.listdir(base_path + 'Color_anonym')
df = pd.read_excel(base_path + 'Datenliste pAVK 2.2 MW.xlsx', sheet_name='f_R', header=[0, 1, 2, 3], engine='openpyxl')
df.head()

Unnamed: 0_level_0           Unnamed: 1_level_0 Unnamed: 2_level_0  \
                  ID Group \n(0=control)\n(1=PAD)           Lastname   
  Unnamed: 0_level_2           Unnamed: 1_level_2 Unnamed: 2_level_2   
                  ID                        Group           Lastname   
0            AR0451R                            1             Alheit   
1            AR0451L                            1             Alheit   
2            AB0949R                            1             Anhalt   
3            AB0949L                            1             Anhalt   
4            AM0459R                            0            Avijgan   

  Unnamed: 3_level_0 Unnamed: 4_level_0               Unnamed: 5_level_0  \
           Firstname             Age \n Sex\n(0= masculine)\n(1= female)   
  Unnamed: 3_level_2 Unnamed: 4_level_2               Unnamed: 5_level_2   
           Firstname               Age                               Sex   
0            Reimund                 67                                0   
1            Reimund                 67                                0   
2           Brigitte                 69                                1   
3           Brigitte                 69                                1   
4             Mahanz                 59                                1   

  Unnamed: 6_level_0      BCVA\n logMAR               BCVA  3x3 OCTA _Central  \
                 Eye Unnamed: 7_level_1 Unnamed: 8_level_1 Unnamed: 9_level_1   
  Unnamed: 6_level_2 Unnamed: 7_level_2 Unnamed: 8_level_2               SSI    
                 Eye             LOGMAR               BCVA       SSI_3x3_cent   
0                  R                0.0                1.0               10.0   
1                  L                0.0                1.0               10.0   
2                  R                0.0                1.0                9.0   
3                  L                0.1                0.8               10.0   
4                  R                0.0                1.0               10.0   

   ...                      Drugs                            \
   ...       Unnamed: 118_level_1      Unnamed: 119_level_1   
   ... Marcumar\n( 0=no)\n(1=yes) Insulin\n( 0=no)\n(1=yes)   
   ...                   MARCUMAR                   INSULIN   
0  ...                        0.0                       0.0   
1  ...                        0.0                       0.0   
2  ...                        0.0                       0.0   
3  ...                        0.0                       0.0   
4  ...                        0.0                       0.0   

                                              \
                        Unnamed: 120_level_1   
  Metformin/\nAntidiabetic\n( 0=no)\n(1=yes)   
                                   METFORMIN   
0                                        0.0   
1                                        0.0   
2                                        0.0   
3                                        0.0   
4                                        0.0   

                                                              OCT-A  \
                                               Unnamed: 121_level_1   
  OCT-A 3x3 central\n(0 = good)\n(1 = intermediate)\n(2 = excluded)   
                                                     GRADE_3x3 cent   
0                                                1.0                  
1                                                0.0                  
2                                                0.0                  
3                                                0.0                  
4                                                0.0                  

                                                                      \
                                                Unnamed: 122_level_1   
  OCT-A 3x3 temporal\n(0 = good)\n(1 = intermediate)\n(2 = excluded)   
                                                      GRADE_3x3 temp   
0        

In [10]:
ages = []
seen_patients = []
# Extract ages from known entities
for row in df.itertuples(index=False):
    if row[0] + '.jpg' not in files:
        continue
    if row[0][:-1] in seen_patients:
        continue
        # pass
    ages.append((row[4], row[1], row[5])) # (age, group[0/1], sex[1=male])
    seen_patients.append(row[0][:-1])

# Add ages for additional controls manually (twice for two eyes)
ages.append((89, 0, 0))
#ages.append((89, 0, 0))
ages.append((86, 0, 1))
#ages.append((86, 0, 1))

filtered_ages = [a[0] for a in ages if a[1] == 1] #and a[2] == 0]

print(f'Num examinations: {len(ages)}, mean: {np.mean([a[0] for a in ages])}, std: {np.std([a[0] for a in ages])}')
print(f'Control examinations: {len(filtered_ages)}, mean: {np.mean(filtered_ages)}, std: {np.std(filtered_ages)}')

Num examinations: 77, mean: 69.18181818181819, std: 9.56010008197256
Control examinations: 57, mean: 67.57894736842105, std: 9.540214732257382


In [16]:
# Read the images to be aligned
im1_gray =  cv2.imread("/home/simon/Pictures/comp_res/ST0352R.jpg", 0)
im2_gray =  cv2.imread("/home/simon/Pictures/comp_res/299_ST0352R.jpg", 0)
im2 = cv2.imread("/home/simon/Pictures/comp_res/299_ST0352R.jpg")

# Find size of image1
sz = im1_gray.shape
# Define the motion model
warp_mode = cv2.MOTION_TRANSLATION

# Define 2x3 or 3x3 matrices and initialize the matrix to identity
if warp_mode == cv2.MOTION_HOMOGRAPHY:
    warp_matrix = np.eye(3, 3, dtype=np.float32)
else:
    warp_matrix = np.eye(2, 3, dtype=np.float32)

number_of_iterations = 5000
termination_eps = 1e-10
criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations,  termination_eps)

# Run the ECC algorithm. The results are stored in warp_matrix.
(cc, warp_matrix) = cv2.findTransformECC (im1_gray,im2_gray,warp_matrix, warp_mode, criteria, None)

if warp_mode == cv2.MOTION_HOMOGRAPHY:
    im2_aligned = cv2.warpPerspective (im2_gray, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
else:
    im2_aligned = cv2.warpAffine(im2, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

# Show final results
cv2.namedWindow('Main', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Main', 800, 800)
cv2.imshow("Main", im1_gray)
cv2.waitKey(2000)
cv2.imshow("Main", im2_gray)
cv2.waitKey(2000)
cv2.imshow("Main", im2_aligned)
cv2.waitKey(2000)
cv2.destroyAllWindows()

cv2.imwrite('/home/simon/Pictures/comp_res/299_aligned_ST0352R.jpg', im2_aligned)

True